In [1]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm # to create loadbard in for loop
import os
from datetime import date
import time
import random
import pickle

For now dowload all earning call transcript for a specified ticker

### url to use for apple
https://seekingalpha.com/api/v3/symbols/aapl/transcripts?filter[until]=undefined&id=aapl&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=true&page[size]=20

https://seekingalpha.com/api/v3/symbols/aapl/transcripts?filter[until]=1501632962&id=aapl&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=false&page[size]=20

In [2]:
def create_url_for_scrap(ticker, publish_time_min ):
    # accept publis_time_min = None
    return_per_request  = 20 #the one used by seekingalpha by default
    ticker = ticker.lower()
    publish_time_min = "undefined" if publish_time_min ==None else str(publish_time_min)
    to_return = ("https://seekingalpha.com/api/v3/symbols/"+ticker+"/transcripts?filter[until]="+publish_time_min
                 +"&id="+ticker
                 +"&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=true&page[size]="
                 + str(return_per_request))
    return to_return
def is_earning_call(element_response):
    title = element_response["attributes"]["title"].lower()
    has_earning_in_title = "earning" in title or "earnings" in title
    has_call_in_title = "call" in title
    return element_response["type"]=="transcript" and has_call_in_title and has_earning_in_title
def get_url_id_date_earning_call(element_response):
    date = element_response["attributes"]["publishOn"][:10]
    call_id = element_response["id"]
    url = element_response["links"]["self"]
    return url, call_id, date

In [3]:
# for now scrapp all earning call transcript from one company
ticker = "AAPL" #Apple hospitality REIT, Inc.
html_path = "data/ticker/"+ticker ## todo save file with name time step*
last_time_scrap_path =html_path+ "/last_time_scrap"
if not os.path.exists(html_path):
    os.makedirs(html_path)
    print(f"The new directory {html_path} has been created!")
if os.path.exists(last_time_scrap_path):
    with open(last_time_scrap_path, "r") as file:
        publish_time_min = file.read()
else : 
    publish_time_min = None
if publish_time_min !="all":
    more_data = True
    executable_path = "./chromedriver_linux"
    service = Service(executable_path)
    option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service,options=option)
    id_URLs = []
    id_to_date = {}
    while more_data:
        url = create_url_for_scrap(ticker, publish_time_min)
        driver.get(url)
        json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
        data_responses = json.loads(json_response)
        if "meta" not in data_responses:
            time.sleep(5) # 2 chances to be flag as robot
            driver.get(url)
            json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
            data_responses = json.loads(json_response)
            if "meta" not in data_responses:
                last_publish_time_min = publish_time_min
                break
        publish_time_min = str(data_responses["meta"]["page"]["minmaxPublishOn"]["min"]) # need to be string
        if publish_time_min!= 'None':
            for element in data_responses["data"]:
                if is_earning_call(element):
                    URL, call_id, date = get_url_id_date_earning_call(element)
                    id_URLs.append((call_id,URL))
                    id_to_date[call_id] = date
        else :
            last_publish_time_min = "all"
            more_data = False
        time.sleep(2+random.uniform(0, 1))


The new directory data/ticker/AAPL has been created!


Download HTML files 

In [4]:
domain_name = 'https://seekingalpha.com'
for call_id, URL in tqdm(id_URLs):
    file_path = html_path + "/"+ call_id + ".html"
    with open(file_path, "w") as file :
        url = domain_name+URL
        html =  requests.get(url).text
        file.write(html)

  0%|          | 0/66 [00:00<?, ?it/s]

Save the last scrapped url of the ticker, so that if bot detected can resume

In [5]:
with open(html_path+"/last_time_scrap", "w") as file:
    file.write(last_publish_time_min)


## Check last file got downloaded, i.e. captcha/bot error 

In [6]:
BeautifulSoup(html).get_text()[:2000]

"Apple F1Q06 (Qtr End 12/31/2005) Earnings Conference Call Transcript | Seeking Alpha                    Seeking Alpha - Go to HomepageTranscriptsTechnologyApple F1Q06 (Qtr End 12/31/2005) Earnings Conference Call TranscriptJan. 19, 2006 8:52 AM ETApple Inc. (AAPL)SA Transcripts127.16K FollowersFollow Apple Computer, Inc. (NASDAQ:AAPL) Q1 2006 Financial Results Conference Call January 18th 2006, 5:00 PM. Executives Nancy Paxton, Senior Director, Investor Relations and Corporate Finance Peter Oppenheimer, Chief Financial Officer Timothy (Tim) Cook, Chief Operating Officer Gary Whistler, Corporate Treasurer Analysts Keith Bachman Banc of America Securities. Benjamin Reitzes, UBS. Robert Semple, Credit Suisse First Boston Gene Munster, Piper Jaffray. Rebecca Runkle, Morgan Stanley. Bill Shope, J.P.Morgan. Charles Wolf, Needham & Company. Joel Wagonfeld, First Albany Capital. Richard Chu, Sg. Cowen & Co. Steve Lidberg, Pacific Crest Securities. Shaw Wu, American Technology. Steven Fortuna,

Save date of transcripts

In [7]:
id_to_date_path = "data/ticker/id_to_date"
if not os.path.exists(id_to_date_path):
    file_id_to_date = {}
else:
    with open(id_to_date_path,"rb") as file :
        file_id_to_date = pickle.load(file)
for call_id, date in id_to_date.items():
    file_id_to_date[call_id] = date
    
with open(id_to_date_path,"wb") as file : 
    pickle.dump(file_id_to_date, file, protocol=pickle.HIGHEST_PROTOCOL)
    